In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [7]:
data = pd.read_csv('datasetfinal.csv')
data

,seller_id,product_name,product_id,buyer_id,product_rating,product_price
0,457,40 Set Menu Sehari-hari Hits di Instagram ala ...,30070,1504,4.9,142800
1,457,7 Hari Belajar Drone Photography (Edisi Revisi),30079,1435,5.0,102000
2,457,Akasha : City Hunter - Complete Edition 03,30121,1560,4.9,49300
3,457,AKASHA : FATE/APOCRYPHA 02,30163,271,4.8,38250
4,457,AKASHA : RECORD OF RAGNAROK 03,30219,6,5.0,38250
...,...,...,...,...,...,...
49995,105,Samyang V-AF 24mm T1.9 Lens for Sony FE Samyan...,23658,47,5.0,8979000
49996,135,Godox S60 LED Focusing 3-Light Kit,29682,132,5.0,18849000
49997,134,Memory 128GB SF-M Tough Series UHS-II SDXC Mem...,28802,250,5.0,1599000
49998,4,Insta360 One X3 Sticky Lensguard,20253,248,5.0,539000


Pada data di atas, terdapat beberapa row data yang tidak memiliki rating, maka dari itu dilakukan preprocessing dengan menghapus row yang mengandung no rating

In [8]:
data = data[data['product_rating'] != 'No Rating']
data

,seller_id,product_name,product_id,buyer_id,product_rating,product_price
0,457,40 Set Menu Sehari-hari Hits di Instagram ala ...,30070,1504,4.9,142800
1,457,7 Hari Belajar Drone Photography (Edisi Revisi),30079,1435,5.0,102000
2,457,Akasha : City Hunter - Complete Edition 03,30121,1560,4.9,49300
3,457,AKASHA : FATE/APOCRYPHA 02,30163,271,4.8,38250
4,457,AKASHA : RECORD OF RAGNAROK 03,30219,6,5.0,38250
...,...,...,...,...,...,...
49995,105,Samyang V-AF 24mm T1.9 Lens for Sony FE Samyan...,23658,47,5.0,8979000
49996,135,Godox S60 LED Focusing 3-Light Kit,29682,132,5.0,18849000
49997,134,Memory 128GB SF-M Tough Series UHS-II SDXC Mem...,28802,250,5.0,1599000
49998,4,Insta360 One X3 Sticky Lensguard,20253,248,5.0,539000


In [9]:
data['product_rating'] = data['product_rating'].astype(float)

<ipython-input-9-d712b2dd6b1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['product_rating'] = data['product_rating'].astype(float)


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49539 entries, 0 to 49999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   seller_id       49539 non-null  int64  
 1   product_name    49539 non-null  object 
 2   product_id      49539 non-null  int64  
 3   buyer_id        49539 non-null  int64  
 4   product_rating  49539 non-null  float64
 5   product_price   49539 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 2.6+ MB


In [12]:
# Encoding menggunakan Label Encoding
user_encoder = LabelEncoder()
product_encoder = LabelEncoder()

data['user'] = user_encoder.fit_transform(data['buyer_id'])
data['product'] = product_encoder.fit_transform(data['product_id'])

n_users = data['user'].nunique()
n_products = data['product'].nunique()
n_latent_factors = 50

<ipython-input-12-cb5857709e54>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['user'] = user_encoder.fit_transform(data['buyer_id'])
<ipython-input-12-cb5857709e54>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['product'] = product_encoder.fit_transform(data['product_id'])


In [13]:
from tensorflow.keras.models import load_model
model = load_model('model.h5')
user_embeddings = model.predict(np.arange(n_users))

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
# --- Rekomendasi Berdasarkan Similarity ---
def recommend_by_similar_users(user_id, top_n=10):
    # Cosine Similarity antar Pengguna
    target_user_embedding = user_embeddings[user_id]
    similarities = cosine_similarity([target_user_embedding], user_embeddings)

    # Sort based on Kemiripan
    similar_users = similarities.argsort()[0][::-1][1:top_n + 1]

    # Ambil produk yang diberi rating tinggi oleh pengguna mirip
    recommended_products = set()
    for similar_user in similar_users:
        # Ambil produk yang sudah pernah diberi rating oleh pengguna mirip
        user_data = data[data['user'] == similar_user]
        high_rated_products = user_data[user_data['product_rating'] >= 4.0]['product']

        # Tambahkan produk ke daftar rekomendasi
        recommended_products.update(high_rated_products)

    # Produk yang belum pernah dinilai oleh pengguna target (diasumsikan belum pernah membeli)
    target_user_products = data[data['user'] == user_id]['product']
    recommended_products -= set(target_user_products)

    # Ambil top_n produk
    recommended_products = list(recommended_products)[:top_n]

    # Decode ke product_id asli
    recommended_product_ids = product_encoder.inverse_transform(recommended_products)

    # Ambil nama produk dari data
    recommended_names = []
    for pid in recommended_product_ids:
        # Ambil nama produk pertama yang cocok
        match = data[data['product_id'] == pid]['product_name']
        if not match.empty:
            recommended_names.append(match.values[0])
        else:
            recommended_names.append(f"(nama tidak ditemukan untuk ID {pid})")

    # Result
    print(f"User-Based Recommendations for User {user_id}:")
    for name in recommended_names:
        print(f"- {name}")

In [18]:
# Contoh penggunaan
user_id = 457  # Misalkan kita ingin memberikan rekomendasi untuk pengguna dengan ID 5
top_n = 10
recommended_products = recommend_by_similar_users(user_id, top_n)

User-Based Recommendations for User 457:
- Buku Belenggu Nalar
- Chefmade WK9300 - Marble Decorating Turntable / Meja Putar Dekor Kue
- 6 Pasang Cangkir Kopi Set Motif Olympia Black / Tea Cup Saucer
- PETUALANGAN ANAK NUSANTARA
- Wiper alat pembersih kaca jendela lion star BR-10 glass cleaner
- 6 Pasang Cangkir Set / Cangkir + Saucer Nikura / Cangkir Teh / Cangkir
- Keychron K6-T2 RGB Backlight Aluminum Frame Blue Switch
- Novel Seribu Wajah Ayah (Nurun Ala)
- Canon Ink Pg 745
- Godox FL 6060 / Godox Softbox FL-6060 / FL6060 Softbox
